# gBizINFOの法人データを用いたスタートアップ企業の分析

[gBizINFO REST API 仕様書](https://info.gbiz.go.jp/hojin/swagger-ui/index.html#/gBizINFO%20REST%20API/searchInfo)に記載されている内容を参考にして、gBizInfoのAPIを用いてデータを取得します。

今回は、2019年から2023年に設立された企業情報を取得します。

In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path

# データ取得に用いるライブラリ
from urllib.parse import urljoin
import requests
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET 

# データ操作に用いるライブラリ
import time
import pandas as pd

In [ ]:
load_dotenv()
api_token = os.getenv("gBizINFO_API_ACCESS_TOKEN")

current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch12").resolve()

## gBizINFO API を用いた企業情報の検索

In [ ]:
# 取得したアクセストークンを指定
headers_gbiz = {
    "Accept": "application/json",
    "X-hojinInfo-api-token": api_token
}
url_hojin = "https://info.gbiz.go.jp/hojin/v1/hojin"
founded_year = ",".join(str(n) for n in range(2019, 2024))
params_gbiz = {
    "founded_year": founded_year,
    "limit": "5000"
}
res_hojin = requests.get(url_hojin, params=params_gbiz, headers=headers_gbiz)
out = res_hojin.json()
gbiz_registered_firm = pd.json_normalize(out, record_path=["hojin-infos"])

In [ ]:
gbiz_registered_firm.head()

## 法人番号からスタートアップ企業の数を都道府県ごとに取得する

続いて、先程取得した法人番号を用いて、企業のその他の情報を取得します。

In [ ]:
# 各法人のDataFrameを一時的に格納するためのリスト
hojin_info_df = []
for corp_num in tqdm(gbiz_registered_firm.corporate_number, desc="法人情報取得中"):
    temp_url = f"https://info.gbiz.go.jp/hojin/v1/hojin/{corp_num}"
    try:
        response = requests.get(temp_url, params=params_gbiz, headers=headers_gbiz)
        response.raise_for_status()  # HTTPエラーのチェック
        temp_df = pd.json_normalize(response.json()["hojin-infos"])
        # データフレームをリストに追加
        hojin_info_df.append(temp_df)
        
    except requests.exceptions.RequestException as e:
        print(f"法人番号 {corp_num} の取得に失敗: {e}")
    except KeyError:
        # "hojin-infos" というキーがJSON内に存在しない場合のエラー
        print(f"法人番号 {corp_num} のレスポンス形式が正しくありません。")

# ループ完了後、リストに格納された全てのDataFrameを一度に結合する
if hojin_info_df:
    gbiz_houjin_firm = pd.concat(hojin_info_df, ignore_index=True)
else:
    gbiz_houjin_firm = pd.DataFrame() # 1件も成功しなかった場合

データの中身を確認します。代表取締役の名前や、住所情報などが取得出来ていることが確認出来ます。

In [ ]:
gbiz_houjin_firm.head()

データ同士を接合します。

In [ ]:
gbiz_merged_df = gbiz_registered_firm.merge(
    gbiz_houjin_firm, on='corporate_number' , suffixes=('', '_detail')
)

In [ ]:
gbiz_merged_df.head()

API ではなく、サポートサイトで配布するCSVデータを読み込む場合（1の方法）

In [ ]:
gbiz_merged_df = pd.read_csv(
    data_path / "gbiz_merged_df.csv",
    index_col=0,
    low_memory=False
)

APIではなく、ｇBizINFOサイトからダウンロードするZIPからファイルを読み込む場合（2の方法）

In [ ]:
csv_df = pd.read_csv(
    data_path / "Kihonjoho_SJIS_yyyymmdd.zip",  # (current_dir / "data" / "ch11").resolve() / "Kihonjoho_SJIS_20250801.zip",
    encoding="shift_jis",
    low_memory=False,
    usecols=["法人番号", "法人名", "郵便番号", "本社所在地","ステータス", "設立年月日", "創業年", "最終更新日"],
    parse_dates=["設立年月日", "最終更新日"],
)
trans_map = {
    "法人番号": "corporate_number", "法人名": "name",
     "本社所在地": "location", "郵便番号": "postal_code",
     "ステータス": "status", "最終更新日": "update_date"
}
required_cols = list(trans_map.keys()) + ["founded_year"]
founded_year_combined = csv_df["設立年月日"].dt.year.fillna(csv_df["創業年"]).astype("Int64")
founded_from_2019 = (csv_df["設立年月日"].dt.year >= 2019) | (csv_df["創業年"] >= 2019)
founded_until_2023 = (csv_df["設立年月日"].dt.year < 2024) | (csv_df["創業年"] < 2024)
gbiz_merged_df = (csv_df
    .assign(**{"founded_year": founded_year_combined})
    .loc[csv_df["法人名"].notna() &
        founded_from_2019 & founded_until_2023 &
        (founded_year_combined.isna() | (founded_year_combined >= 2019)),
        required_cols]
    .rename(columns=trans_map)
)

In [ ]:
#gbiz_merged_df.to_csv(data_path / "gbiz_merged_df.csv", index=False)
#gbiz_merged_df = pd.read_csv(data_path / "gbiz_merged_df.csv")

## 取得したスタートアップ企業のデータを確認する

さて、ここで都道府県ごとにどの程度スタートアップが存在するのかをテストしたいと思います。
ここでは、正規表現を用いて先ほどの住所情報から都道府県名を取り出すことにします。

In [ ]:
firms_with_prefecture = gbiz_merged_df.assign(**{
    "都道府県": lambda df: df["location"]
        .str.extract(r'^(東京都|北海道|(?:京都|大阪)府|.{2,3}県)', expand=False)
        .fillna("")
})

In [ ]:
firms_with_prefecture["都道府県"].unique()

得られた都道府県の情報を用いて、過去4年に開業した企業の都道府県ごとの数について確認しましょう。

In [ ]:
firms_with_prefecture.groupby('都道府県').size().sort_values(ascending=False)

やはり、東京都にスタートアップが多く存在することがわかります。

なお、これらのスタートアップがデータ取得の段階でも存在しているかどうかの確認をしたいと思います。ここでは、statusを用います。

In [ ]:
firms_with_prefecture.groupby('status').size().sort_values(ascending=False)

## スタートアップ企業の立地を地図上にプロットする

- [シンプル ジオコーディング 実験](https://geocode.csis.u-tokyo.ac.jp/home/simple-geocoding/)
- [シンプル ジオコーディング 実験 参加規約](https://geocode.csis.u-tokyo.ac.jp/home/simple-geocoding/simple-geocoding-tems-of-use/)


緯度・経度の付与作業には、使用するインターネット環境や端末によっては、80分から100分程度の時間を要します。そのため、緯度・経度情報を付与したCSVデータをサポートサイト上で配布します（CSVデータの読み込みの方法は後述します）。

CSVを用いる場合、以下のセルはスキップしてください。

In [ ]:
# tqdmをpandasで使えるように初期化する
tqdm.pandas(desc="ジオコーディング中...")

def geocode_address(address):
    base_url = 'https://geocode.csis.u-tokyo.ac.jp/cgi-bin/simple_geocode.cgi?addr='
    try:
        # タイムアウトを設定し、無限に待機するのを防ぐ
        response = requests.get(base_url + address, timeout=10)
        response.raise_for_status()  
        # レスポンスはXML形式のため、XMLとしてパースする
        root = ET.fromstring(response.content)
        # <candidate> タグが存在するかチェックする
        candidate = root.find('candidate')
        if candidate is not None:
            # <longitude> と <latitude> タグから経度と緯度を取得する
            lon = candidate.find('longitude').text
            lat = candidate.find('latitude').text
            # APIサーバーへの負荷を考慮し、1秒間待機させる
            time.sleep(1)
            # 経度と緯度をfloat型に変換して返す
            return pd.Series([float(lon), float(lat)], index=['longitude', 'latitude'])
        else:
            return pd.Series([None, None], index=['longitude', 'latitude'])

    except (requests.exceptions.RequestException, IndexError, KeyError) as e:
        print(f"住所 '{address}' の処理中にエラーが発生しました: {e}")
        return pd.Series([None, None])
    
# 元のDataFrameをコピーして、新しいDataFrame firms_with_prefecture_with_geocode を作成する
firms_with_prefecture_with_geocode = firms_with_prefecture.copy()

# 'location_x'列の各住所に対してジオコーディング関数を適用し、結果をfirms_with_prefecture_with_geocodeに新しい列として追加する
# .progress_applyを使用すると進捗状況が表示される
firms_with_prefecture_with_geocode[['longitude', 'latitude']] = (
    firms_with_prefecture_with_geocode['location']
    .progress_apply(geocode_address)
)

In [ ]:
# 結果を表示
firms_with_prefecture_with_geocode.head()

配布データを使用する場合はこちらを実行する

In [ ]:
firms_with_prefecture_with_geocode.to_csv(data_path / "gbizinfo_list.csv", index=False)

In [ ]:
firms_with_prefecture_with_geocode = pd.read_csv(
    data_path / "gbizinfo_list.csv", index_col=0, low_memory=False
)

In [ ]:
firms_with_prefecture_with_geocode[['longitude', 'latitude']].head()

In [ ]:
import folium
# ヒートマップのプラグインをインポート
from folium.plugins import HeatMap

#NaNの場合値を除去し、latlng_dropna　に保存する
latlng_dropna=firms_with_prefecture_with_geocode[['latitude', 'longitude']].dropna().values.tolist()

# 東京駅の緯度と経度を直接指定
# [緯度, 経度] の順でリストを作成する
tokyo_station_coords = (35.681236, 139.767125)

# 緯度・経度を指定して地図の中心を設定する
tokyo_map = folium.Map(location=tokyo_station_coords, zoom_start=11)

# 緯度・経度と場所情報をヒートマップとしてプロット
HeatMap(latlng_dropna, radius=11, blur=10).add_to(tokyo_map)

In [ ]:
tokyo_map